In [ ]:
from utils.constants import BigMarketsSubset, Correspondence, SmallMarketsSubset
from data.API_KEYS import chart_studio_key
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from math import log
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls


username = "abiola_adeye"
api_key = chart_studio_key

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [ ]:
def to_real_market_size(elem):
    if "M" in elem:
        return float(elem[:-1])
    if "K" in elem:
        return float(elem[:-1]) / 1000
    else:
        return np.nan


market_size_df = pd.read_csv("./data/market_sizes.csv")
market_size_df["TV MARKET SIZE"] = market_size_df["TV MARKET SIZE"].apply(to_real_market_size)
market_size_df["METRO POPULATION"] = market_size_df["METRO POPULATION"].apply(lambda x: x / (10**6))
market_size_df["MARKET TYPE"] = market_size_df["TV MARKET SIZE"].apply(
    lambda x: "BIG MARKET" if x > 2 else "SMALL MARKET" if x < 1.5 else "MEDIUM MARKET"
)

In [ ]:
market_size_df

In [ ]:
tv_market_size = market_size_df["TV MARKET SIZE"]
population = market_size_df["METRO POPULATION"]
team_name = market_size_df["TEAM"]

# Choose a font
font = fm.FontProperties(family="Ubuntu", size=20, weight="bold")
label_font = fm.FontProperties(family="Ubuntu", size=12, style="italic")
market_font = fm.FontProperties(family="Ubuntu", size=16, weight="bold")


# Set figure size and create subplot
fig, ax = plt.subplots(figsize=(18, 10))
ax.scatter(tv_market_size, population, s=population * tv_market_size * 10, c=market_size_df.index)
ax.axvline(x=2, color="r", linestyle="-")
ax.axvline(x=1.5, color="r", linestyle="-")

# Add gridlines
ax.grid(True, linestyle="-", linewidth=0.5, color="lightgray")

for i in range(len(population)):
    ax.annotate(market_size_df["TEAM"].iloc[i], (tv_market_size.iloc[i] * 1.03, population.iloc[i] * 1.02), font=label_font)

# Add title and axis labels
ax.set_title("TV Market Size vs Metro Population", fontproperties=font)
ax.set_xlabel("TV Market Size", fontproperties=font)
ax.set_ylabel("Metro Population", fontproperties=font)


# Add labels for market sizes
ax.annotate("SMALL MARKET", (0.75, 17.5), font=market_font)
ax.annotate("BIG MARKET", (2.4, 17.5), font=market_font)

# Use a logarithmic scale for both axes
ax.set_xscale("log")
ax.set_yscale("log")

In [ ]:
import plotly.express as px

# Set figure size and create subplot
fig = px.scatter(
    market_size_df,
    x="TV MARKET SIZE",
    y="METRO POPULATION",
    custom_data=["TEAM"],
    color="MARKET TYPE",
    size=market_size_df["METRO POPULATION"],
    log_x=True,
    log_y=False,
)

#
# Add vertical lines
fig.add_shape(type="line", x0=2, x1=2, y0=0, y1=1, name="1.5M", yref="paper", line=dict(color="red", width=2))
fig.add_shape(type="line", x0=1.5, x1=1.5, y0=0, y1=1, yref="paper", line=dict(color="red", width=2))


# Add title and axis labels
fig.update_layout(
    xaxis_title="TV Market Size (in millions)",
    xaxis_title_font=dict(family="Ubuntu", size=12),
    yaxis_title="Metro Population (in millions)",
    yaxis_title_font=dict(family="Ubuntu", size=12),
)

# Add hover labels for team names
fig.update_traces(hovertemplate="<b>%{customdata[0]}</b>")
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Ubuntu",
    )
)
fig.update_layout(yaxis_range=[0, 15])
fig.update_layout(xaxis_range=[0, 0.8])


# fig.update_layout(hovermode='x unified')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=0.8))
fig.show()

fig.write_html("data/team_market_size_comparison.html")

py.plot(fig, filename="market_size_comparison", auto_open=True)

In [ ]:
import plotly

plotly.offline.init_notebook_mode()